In [ ]:
import os
import copy

import nbimporter

import utils.file_manager as fm
import utils.pre_processing as ppc

import minetext.clustering.distance as distance
import minetext.clustering.kmedoids as kmedoids

In [ ]:
# Diretórios base
data_basedir = os.path.join(os.getcwd(), "data") # Diretório geral de dados
exports_basedir = os.path.join(data_basedir, "exports") # Subdiretório dos exports
clean_exports_basedir = os.path.join(data_basedir, "clean_exports") # Subdiretório os exports pré-processados
results_basedir = os.path.join(data_basedir, "results") # Subdiretório dos resultados

# Cria os diretórios do workspace
fm.create_directory(data_basedir)
fm.create_directory(exports_basedir)
fm.create_directory(clean_exports_basedir)
fm.create_directory(results_basedir)

# Lista dos nomes dos diretórios que contém os exports
exports_directories = ["facebook", "telegram"]

# Colunas dos exports a serem consideradas
columns = ["Content", "CleanContent", "Source", "Rating"]

jaccard_calculator = distance.JaccardCalculatorDistance()

In [ ]:
# Realiza o pré-processamento e armazenamento dos exports
for directory in exports_directories:
    filename = directory + ".csv"
    
    export_dirpath = os.path.join(exports_basedir, directory)
    
    # Lê os exports do diretório
    export = fm.read_csv_directory(dirpath=export_dirpath, header=1)
    
    # Realiza o pré-processamento do conteúdo dos exports
    clean_export = ppc.clean_reviews(export=export)
    
    # Salva os exports já pré-processados em um arquivo .csv
    fm.data_to_csv(data=clean_export,\
                   dirpath=clean_exports_basedir,\
                   filename=filename,\
                   index=False,\
                   columns=columns)

In [ ]:
# Função que transforma uma tupla do dataframe em um dict()
# A função considera os atributos: Content, CleanContent, Source e Rating
def tuple_to_dict(tuple_t, columns, indexes=None):
    enumeration = None
    
    if not indexes:
        enumeration = enumerate(columns)
    else:
        enumeration = zip(indexes, columns)
    
    review = dict()
    
    for index, column in enumeration:
        review[column] = tuple_t[index]
        
    return review

In [ ]:
for directory in exports_directories:
    clean_exports_filename = directory + ".csv"
    
    clean_export = fm.read_csv_file(dirpath=clean_exports_basedir,\
                                   filename=clean_exports_filename)
    
    export_reviews = clean_export.itertuples() # Traz as linhas do dataframe em formato de tupla
    columns = clean_export.keys()
    indexes = range(1, len(columns))
    
    # Dict que armazenará as reviews como documentos a serem clusterizados
    reviews = dict()
    reviews["reviews"] = []
    
    for export_review in export_reviews:
        review = tuple_to_dict(export_review, columns, indexes)
        
        reviews["reviews"].append(review)
    
    results = dict()
    results["k"] = []
    results["iteractions"] = []
    results["sse"] = []
    results["clusters"] = []
        
    for k in range(2, 16):
        for itr in range(1, 6):
            # Simulação de passagem de parâmetro po cópia.
            # Necessário pois é preciso uma cópia para cada instância do kmedoids, para evitar inconsistências.
            documents = copy.copy(reviews["reviews"])
            
            kmedoids_instance = kmedoids.Kmedoids(k=k,\
                                              documents=documents,\
                                              distance_calculator=jaccard_calculator,\
                                              collection_field="reviews", text_field_name="CleanContent")
        
            kmedoids_instance.clustering()
            
            results["k"].append(k)
            results["sse"].append(kmedoids_instance.calculate_sse())
            results["iteractions"].append(itr)
            results["clusters"].append(kmedoids_instance.get_clusters())
                
    results_dirpath = os.path.join(results_basedir, directory)
    overall_results_filename = "overall_results.csv"
    
    fm.data_to_csv(data=results,\
                   filename=overall_results_filename,\
                   dirpath=results_dirpath,\
                   columns=["k", "sse", "iteractions"])
    
    fm.clusters_to_csv(results=results,\
                       dirpath=results_dirpath,\
                       collection_field="reviews")